[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/21--no-sql-database-mongo/06--document-database_mongodb--data-analysis.ipynb)

In [ ]:
!sudo apt install mongodb
!sudo service mongodb start

In [ ]:
!mkdir -p var
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson -O var/video_movies.bson
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movieDetails.bson -O var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection movies var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection films var/video_movies.bson

Aggregation framework

```json
{
     "_id" : ObjectId("5b107bec1d2952d0da904dd7"),
     "title" : "Titan A.E.",
     "year" : 2000,
     "rated" : "PG",
     "runtime" : 94,
     "countries" : [
             "USA"
     ],
     "genres" : [
             "Animation",
             "Action",
             "Adventure"
     ],
     "director" : "Don Bluth, Gary Goldman, Art Vitello",
     "writers" : [
             "Hans Bauer",
             "Randall McCormick",
             "Ben Edlund",
             "John August",
             "Joss Whedon"
     ],
     "actors" : [
             "Matt Damon",
             "Bill Pullman",
             "John Leguizamo",
             "Nathan Lane"
     ],
     "plot" : "A young man learns that he has to find a hidden Earth ship before an enemy alien species does in order to secure the survival of humanity.",
     "poster" : "http://ia.media-imdb.com/images/M/MV5BMjE0NTU0ODg4NV5BMl5BanBnXkFtZTcwNzY3MTQyMQ@@._V1_SX300.jpg",
     "imdb" : {
             "id" : "tt0120913",
             "rating" : 6.6,
             "votes" : 50875
     },
     "tomato" : {
             "meter" : 52,
             "image" : "rotten",
             "rating" : 5.7,
             "reviews" : 99,
             "fresh" : 51,
             "consensus" : "Great visuals, but the story feels like a cut-and-paste job of other sci-fi movies.",
             "userMeter" : 60,
             "userRating" : 3.2,
             "userReviews" : 69055
     },
     "metacritic" : 48,
     "awards" : {
             "wins" : 1,
             "nominations" : 7,
             "text" : "1 win & 7 nominations."
     },
     "type" : "movie"
}
```

In [ ]:
#Connect to database
!pip install pymongo

In [ ]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://localhost:27017')

In [ ]:
db = client.datascience

agregation operators
----------------------

* \$project -> schape


```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "imdb" : {
             "id" : "tt0120913",
             "rating" : 6.6,
             "votes" : 50875
     }
}
```

```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "rating" : 6.6
}
```

* \$match -> filtering
* \$group
* \$sort
* \$skip
* \$limit
* \$unwind

```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : [
     "Matt Damon",
     "Bill Pullman",
     "John Leguizamo"
 ],
}
```
unwind result in
```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : "Matt Damon"
},
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : "Bill Pullman"
},
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : "John Leguizamo"
}
```


Which movie has the highest nomination to winning ratio?
----------------------------------------------------

In [ ]:
pp(db.movies.find().next())

In [ ]:
pp(list(db.movies.aggregate([
    {"$match": {"awards": {"$exists": "1"}}},
    {"$match": {"awards.wins": {"$gt": 0}, "awards.nominations": {"$gt": 0}}},
    {
        "$project": {
            "title": 1,
            "awards": 1,
            'rating': '$imdb.rating',
            "ratio": {
                "$divide": ['$awards.wins', '$awards.nominations']
            }
        }
    },
    {"$sort": {"ratio": -1}},
    {"$limit": 2}
])))

Which actor has participated in the highest amount of movies?
-------------------------------------------------------------


In [ ]:
#single movie
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors"}},
    {"$sort": {"rating": -1}},
    {"$limit": 5}
])))

In [ ]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors"}},
    {"$group": {"_id": "$actor", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
])))

Whoo hoo Tom hanks and Natalie Portman taking the lead

Which actor had highest average movie rating value?
----------------------------------------------------

In [ ]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors", "rating": "$imdb.rating"}},
    {"$group": {"_id": "$actor", "avg_rating": {"$avg": "$rating"}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])))

hmm, are they one movie star?!

In [ ]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors", "rating": "$imdb.rating"}},
    {"$group": {"_id": "$actor", "avg_rating": {"$avg": "$rating"}, "count": {"$sum": 1}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])))

indeed, avg trap, lets check with at least 2 movies

In [ ]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors", "rating": "$imdb.rating"}},
    {"$group": {
        "_id": "$actor",
        "avg_rating": {
            "$avg": "$rating"
        },
        "count": {
            "$sum": 1
        },
        "max_score": {
            "$max": "$rating"
        }
    }},
    {"$match": {"count": {"$gt": 2}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])))

Hmm Harrison Ford !!! 

![](../images/ford.jpg)

Grouping operators
------------------
* \$first
* \$last
* \$max
* \$min
* \$avg